# NO₂ Visualization for Göd (2020–2024)

Clean, consolidated notebook to generate consistent maps and animations from existing GeoTIFFs for the Göd area.

This notebook:
- Loads annual and quarterly NO₂ GeoTIFFs
- Renders styled maps with administrative boundaries and plant marker
- Produces PNG frames and animated GIFs with a consistent color scale

Requirements: `rasterio`, `geopandas`, `matplotlib`, `imageio`, `shapely`



In [ ]:
import os
import re
import glob
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from shapely.geometry import Point
import matplotlib.patheffects as path_effects


In [ ]:
# Configuration
ANNUAL_TIFS = {
    "2020": "NO2_June2020_God_FIXED.tif",
    "2022": "NO2_June2022_God_FIXED.tif",
    "2024": "NO2_June2024_God_FIXED.tif",
}

QUARTERLY_DIR = "quarterly"
SHAPEFILE = "gadm41_HUN_2.shp"
PLANT_COORDS = (19.167085, 47.679723)  # (lon, lat)

OUTPUT_ANNUAL = os.path.join("output", "annual")
OUTPUT_QUARTERLY = os.path.join("output", "quarterly")
os.makedirs(OUTPUT_ANNUAL, exist_ok=True)
os.makedirs(OUTPUT_QUARTERLY, exist_ok=True)

# Load base layers once
BASE_GDF = gpd.read_file(SHAPEFILE)
BASE_GDF = BASE_GDF[BASE_GDF.is_valid & ~BASE_GDF.is_empty].copy()
PLANT_GDF = gpd.GeoDataFrame(geometry=[Point(PLANT_COORDS)], crs="EPSG:4326")


In [ ]:
def compute_global_scale(tif_paths):
    vmin, vmax = float("inf"), float("-inf")
    for path in tif_paths:
        with rasterio.open(path) as src:
            data = src.read(1)
            valid = (data > 0) & ~np.isnan(data)
            if np.any(valid):
                vmin = min(vmin, float(np.min(data[valid])))
                vmax = max(vmax, float(np.max(data[valid])))
    if not np.isfinite(vmin) or not np.isfinite(vmax):
        # sensible defaults seen in notebook
        vmin, vmax = 1.8e-5, 7.0e-5
    return vmin, vmax


def plot_no2_with_context(no2, extent, raster_crs, title, vmin, vmax, output_png):
    gdf_proj = BASE_GDF.to_crs(raster_crs)
    plant_proj = PLANT_GDF.to_crs(raster_crs)
    x, y = plant_proj.geometry[0].x, plant_proj.geometry[0].y

    fig, ax = plt.subplots(figsize=(10, 8))
    img = ax.imshow(no2, cmap="plasma", extent=extent, vmin=vmin, vmax=vmax)
    gdf_proj.boundary.plot(ax=ax, edgecolor="white", linewidth=1)
    plant_proj.plot(ax=ax, color="cyan", marker="o", markersize=100, edgecolor="black", zorder=5)

    label = ax.text(x + 0.01, y + 0.01, "Göd", fontsize=12, color="white", weight="bold")
    label.set_path_effects([path_effects.Stroke(linewidth=2, foreground="black"), path_effects.Normal()])

    ax.set_xlim(x - 0.15, x + 0.15)
    ax.set_ylim(y - 0.15, y + 0.15)

    plt.title(title, fontsize=14)
    cbar = plt.colorbar(img, ax=ax, fraction=0.03, pad=0.04)
    cbar.set_label("mol/m²")
    plt.tight_layout()
    plt.savefig(output_png, dpi=150)
    plt.close()
    return output_png


def build_gif(frame_paths, gif_path, duration=2.5, loop=0):
    with imageio.get_writer(gif_path, mode="I", duration=duration, loop=loop) as writer:
        for frame in frame_paths:
            writer.append_data(imageio.imread(frame))
    return gif_path


In [ ]:
# Annual maps (2020, 2022, 2024)
annual_paths = [ANNUAL_TIFS[y] for y in sorted(ANNUAL_TIFS.keys())]
vmin, vmax = compute_global_scale(annual_paths)

annual_frames = []
for year in sorted(ANNUAL_TIFS.keys()):
    tif_path = ANNUAL_TIFS[year]
    with rasterio.open(tif_path) as src:
        no2 = src.read(1)
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
        raster_crs = src.crs
    title = f"NO₂ Concentration Over Göd – June {year}"
    out_png = os.path.join(OUTPUT_ANNUAL, f"NO2_God_{year}.png")
    annual_frames.append(
        plot_no2_with_context(no2, extent, raster_crs, title, vmin, vmax, out_png)
    )

gif_annual = os.path.join(OUTPUT_ANNUAL, "NO2_God_Annual.gif")
build_gif(annual_frames, gif_annual, duration=2.5, loop=0)
gif_annual


In [ ]:
# Quarterly series (2022–2024)
# Collect all quarterly TIFs with proper sorting (Q1_2022 ... Q4_2024)
import re

def sort_key(fname):
    m = re.search(r"Q([1-4])[_]?(\d{4})", fname)
    if m:
        q = int(m.group(1))
        y = int(m.group(2))
        return y * 4 + q
    return 10**9

quarterly_tifs = sorted([
    os.path.join(QUARTERLY_DIR, f)
    for f in os.listdir(QUARTERLY_DIR)
    if f.endswith(".tif") and "God" in f
], key=sort_key)

vmin_q, vmax_q = compute_global_scale(quarterly_tifs)

quarterly_frames = []
for tif_path in quarterly_tifs:
    with rasterio.open(tif_path) as src:
        no2 = src.read(1)
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
        raster_crs = src.crs
    # derive title from filename
    base = os.path.basename(tif_path).replace("NO2_", "").replace("_God.tif", "").replace("_", " ")
    title = f"NO₂ Concentration Over Göd – {base}"
    out_png = os.path.join(OUTPUT_QUARTERLY, os.path.basename(tif_path).replace('.tif', '.png'))
    quarterly_frames.append(
        plot_no2_with_context(no2, extent, raster_crs, title, vmin_q, vmax_q, out_png)
    )

# Build GIF
gif_quarterly = os.path.join(OUTPUT_QUARTERLY, "NO2_God_Quarterly.gif")
build_gif(quarterly_frames, gif_quarterly, duration=5.0, loop=0)
gif_quarterly
